In [ ]:
# jupyter nbconvert --to python test_software_dev_version.ipynb test_software_dev_version.py
%reset -f
import importlib
import numpy as np
import matplotlib.pyplot as plt

import sys

import Frog
importlib.reload(Frog)
import Frog.toolbox as toolbox
import Frog.frog_data_analysis as frog_data_analysis

import pickle
import time
from scipy.optimize import curve_fit


# Load the results

In [ ]:
# The directory where is located the FROG results:
directory = './'

GP, L_moleculetype_result = frog_data_analysis.load_result(directory, name_result='L_moleculetype_result.p', what_to_print=['general info', 'diagram info'])


# Individual value

The axis_space contains the axis_space.value and axis_space.unit which are a list ready for plotting and the unit used respectively. The axis depend on the type of space-discretization used. 

* Averaged: 
> No space discretization is used. Therefore thus axis is not usefull.

* Plane_ij: 
> The space discretization is made along the laboratory axis k. The `axis_space.value` is a list of N elements (N being the number of bin you have defined to discretize the space). You can check this N by: `my_diagram.size[0]`, `my_diagram_sdparameter.bin_size[0]` and `len(my_diagram.axis_space.value)`. The values goes though the box size widith in the discretization direction using N values. Please note that the box size evolution during the simulation is taken into account in the molecule's attribution to each slices. However, the `my_diagram.axis_space.value` is made using the first frame. 

* Layer:
> The space discretization is made over the layer attribution using the module pytim. See the tutorial about this scheme for more details. In short, `my_diagram.axis_space` contains the layer numbering for this diagram. If 4 layer are used for this diagram, `my_diagram.axis_space` goes from -4 to 4 (9 elements in total). 

The unit are built in by Frog. To show the unit of an axis, use: `axis.unit.print_unit()`
You can change the unit for a diagram, see below. 

## Plane

In [ ]:
# To load 
MT_name = 'Water_TIP4P2005'
name_diagram = 'density_slice_z'
my_diagram = frog_data_analysis.return_diagram(GP, L_moleculetype_result, MT_name, name_diagram)

# To have access to its diagram parameter object:
for KKK in range(0, len(L_moleculetype_result), 1): # finding the MT 
    if L_moleculetype_result[KKK].name == MT_name:
        K_MT = KKK

for k in range(0, len(L_moleculetype_result[K_MT].mtparameter.dparameter.L_diagram), 1): # finding the sdparameter relative to the diagram asked
    diagram_name = L_moleculetype_result[K_MT].mtparameter.dparameter.L_diagram[k].name
    if diagram_name == name_diagram:
        my_diagram_sdparameter = L_moleculetype_result[K_MT].mtparameter.dparameter.L_diagram[k]
        
# In the Plane_ij case: 
L_xyz = ['x', 'y', 'z'] # my_diagram_sdparameter.discretization_type stores the discretized laboratory axis using this correspondance 
print('The number of bin used to discretize the ' + L_xyz[my_diagram_sdparameter.discretization_type] + ' laboratory axis is:',
      my_diagram.size[0])
print('Note that these values are the same:', my_diagram.size[0], my_diagram_sdparameter.bin_size[0], len(my_diagram.axis_space.value))
print('The box size in the ' + L_xyz[my_diagram_sdparameter.discretization_type] + 'laboratory axis is', GP.box_size[my_diagram_sdparameter.discretization_type])
print('The my_diagram.axis_space.value is:', my_diagram.axis_space.value)
print('The my_diagram.axis_space unit is:', my_diagram.axis_space.unit.print_unit())


An example: let's see in which slice will be attributed the molecule number 1 at the first time step:  

In [ ]:
# Example of discretization:
Axis_of_discretization = 2 # Z laboratory axixs

mean_position = L_moleculetype_result[0].L_molecule[0].mean_position
z_mean_position = mean_position[Axis_of_discretization]
nbr_bits = my_diagram.size[0]
y_min = 0
y_max = GP.box_size[2]

print('Box size along the z direction:', GP.box_size[2])
print('Number of bin to discretize the observable "mean position" along the z-axis:', my_diagram.size[0])
print('The mean position ' + str(z_mean_position) + ' correspond to the bin number:', toolbox.binarize_array(z_mean_position, nbr_bits, y_min, y_max, pbc=False))

## Layer

In [ ]:
# To load 
MT_name = 'Water_TIP4P2005'
name_diagram = 'density_layer'
my_diagram = frog_data_analysis.return_diagram(GP, L_moleculetype_result, MT_name, name_diagram)

# To have access to its diagram parameter object:
for KKK in range(0, len(L_moleculetype_result), 1): # finding the MT 
    if L_moleculetype_result[KKK].name == MT_name:
        K_MT = KKK

for k in range(0, len(L_moleculetype_result[K_MT].mtparameter.dparameter.L_diagram), 1): # finding the sdparameter relative to the diagram asked
    diagram_name = L_moleculetype_result[K_MT].mtparameter.dparameter.L_diagram[k].name
    if diagram_name == name_diagram:
        my_diagram_sdparameter = L_moleculetype_result[K_MT].mtparameter.dparameter.L_diagram[k]
        
# In the Layer case: 

print('The number of layer used to discretize all the system is ' + str(my_diagram_sdparameter.bin_size[0]) + ' which correspond to 2N+1, N being the input in the parameter file. N layer are used for the bottom interface, N for the upper, and 1 for the bulk-like phase.')
print('Note that these values are the same:', my_diagram.size[0], my_diagram_sdparameter.bin_size[0], len(my_diagram.axis_space.value))
print('The my_diagram.axis_space.value is:', my_diagram.axis_space.value, 'The negative value are for the lower interface, the positive for the upper interface. 0 stands for the bulk-like area.')
print('The my_diagram.axis_space unit is:', my_diagram.axis_space.unit.print_unit())


Contrarilly to the Plane geometrical selection, the layer attribution is made only once with the higest possible layer required.

If the available layer number is from -6 to +6, the layer number 0 is the bulk phase. The layer -6 is the most outter one in the bottom part of the MD box, the layer +6 the most outter in the upper part. The layer -1 is very like the 'bulk' in the lower part, the layer +1 is very like to the 'bulk' in the upper part. If enough layer is used, the layer -1 and 1 would represent the same one. If the number of layer goes from N to N+1, the layer number i becomes the layer number i+1. If the number of layer goes from N to N-1, the layer number -1 and 1 becomes 0: they are no longer considerated as in the interface part, but in the bulk. 

In this run, here are all the diagram that are using the layer attribution, and the number of layer used for each diagram


In [ ]:
print('Diagram name,', 'Number of layer per interface')
for k in range(0, len(L_moleculetype_result[0].mtparameter.dparameter.L_diagram), 1): 
    diagram_name = L_moleculetype_result[0].mtparameter.dparameter.L_diagram[k].name
    my_diagram_sdparameter = L_moleculetype_result[K_MT].mtparameter.dparameter.L_diagram[k]
    if my_diagram_sdparameter.discretization_type == 10: #layer type geometrical selection
        print(diagram_name, int((my_diagram_sdparameter.bin_size[0]-1)/2))
        
print('The maximal number of layer required is:', GP.layer_nbr_max)

Therefore, the layer attribution for all the molecule are made using the maximal number of layer required, using the `GP.layer_nbr_max` parameter (not user defined). 

Then, during the diagram attribution, the molecule layer number is changed depending on the number of layer used for this diagram. For instance, if a molecule has for layer number +1 using 6 layer, its layer number will be 0 for all the diagram that uses 5 layer or less. 

An example: let's see in which layer will be attributed the molecule number 1 at the first time step:  

In [ ]:
# Example of discretization:

mean_position = L_moleculetype_result[0].L_molecule[0].mean_position
layer_number = L_moleculetype_result[0].L_molecule[0].layer

print('The mean position ' + str(mean_position) + ' correspond to the layer number:', layer_number)

# Density

## Plane discretization

In [ ]:
# To load 
MT_name = 'Water_TIP4P2005'
name_diagram = 'density_slice_z'
my_diagram = frog_data_analysis.return_diagram(GP, L_moleculetype_result, MT_name, name_diagram)
my_diagram.switch_unit_diagram('axis_space', 'length', 'nm', custom_change=False, molar_mass=False)
my_diagram.switch_unit_diagram('distribution', 'population', 'kg', custom_change=False, molar_mass=18)
my_diagram.switch_unit_diagram('distribution', 'length', 'dm')

density_plane_z = my_diagram.value/GP.nbr_time_step

In [ ]:
plt.figure(1, figsize=(10, 7))
plt.plot(my_diagram.axis_space.value, my_diagram.value/GP.nbr_time_step, lw=3)

plt.text(my_diagram.axis_space.value[0], max(my_diagram.value/GP.nbr_time_step)*1.1, 'Gas', fontsize=25, ha='center')
plt.text(my_diagram.axis_space.value[-1], max(my_diagram.value/GP.nbr_time_step)*1.1, 'Gas', fontsize=25, ha='center')
plt.text(my_diagram.axis_space.value[int(len(my_diagram.axis_space.value)/2)], max(my_diagram.value/GP.nbr_time_step)*1.1, 'Bulk', fontsize=25, ha='center')
plt.xlabel(r'Altitude [$' + my_diagram.axis_space.unit.print_unit() + ']$', fontsize=25)
plt.ylabel(r'Density [$' + my_diagram.unit.print_unit() + ']$', fontsize=25)
plt.ylim([0, max(my_diagram.value/GP.nbr_time_step)*1.2])
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

## Layer discretization

In [ ]:
# To load 
MT_name = 'Water_TIP4P2005'
name_diagram = 'density_layer'
my_diagram = frog_data_analysis.return_diagram(GP, L_moleculetype_result, MT_name, name_diagram)

L_density_layer_x = my_diagram.axis_space.value
L_density_layer = my_diagram.value # here is printed the numbe of molecule found in the relative layer, not a density (ie not [molecule/volume], juste [molecule])

In [ ]:
plt.figure(1, figsize=(10, 7))
plt.plot(my_diagram.axis_space.value, my_diagram.value, lw=3)

plt.text(my_diagram.axis_space.value[0], max(my_diagram.value)*1.1, 'Gas', fontsize=25, ha='center')
plt.text(my_diagram.axis_space.value[-1], max(my_diagram.value)*1.1, 'Gas', fontsize=25, ha='center')
plt.text(0, max(my_diagram.value)*1.1, 'Bulk', fontsize=25, ha='center')
plt.xlabel(r'Altitude [$' + my_diagram.axis_space.unit.print_unit() + ']$', fontsize=25)
plt.ylabel(r'Number of molecule found', fontsize=25)
plt.ylim([0, max(my_diagram.value)*1.2])
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

# Molecular Orientation

## Plane 

In [ ]:
# To load 
MT_name = 'Water_TIP4P2005'
name_diagram = 'molecular_orientation_slice_z'
my_diagram = frog_data_analysis.return_diagram(GP, L_moleculetype_result, MT_name, name_diagram)

In [ ]:
plt.figure(1, figsize=(10, 7))
plt.plot(my_diagram.axis_space.value, my_diagram.mean.T[2], lw=3)

#plt.xlim([85, 105])
plt.xlabel(r'Altitude [$' + my_diagram.axis_space.unit.print_unit() + ']$', fontsize=25)
plt.ylabel(r'Molecular Orientation ', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

## Layer

In [ ]:
# To load 
MT_name = 'Water_TIP4P2005'
name_diagram = 'molecular_orientation_layer'
my_diagram = frog_data_analysis.return_diagram(GP, L_moleculetype_result, MT_name, name_diagram)

In [ ]:
plt.figure(1, figsize=(10, 7))
plt.plot(my_diagram.axis_space.value, my_diagram.mean.T[2], lw=3)

#plt.xlim([85, 105])
plt.xlabel(r'Altitude [$' + my_diagram.axis_space.unit.print_unit() + ']$', fontsize=25)
plt.ylabel(r'Molecular Orientation ', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

# H-bond

## Plane

In [ ]:
# To load 
MT_name = 'Water_TIP4P2005'
name_diagram = 'hbond_slice_z_Water_TIP4P2005'
my_diagram = frog_data_analysis.return_diagram(GP, L_moleculetype_result, MT_name, name_diagram)

In [ ]:
plt.figure(1, figsize=(10, 7))
plt.plot(my_diagram.axis_space.value, my_diagram.mean.T[0], lw=3, label='Own')
plt.plot(my_diagram.axis_space.value, my_diagram.mean.T[1], lw=3, label='Given')
plt.xlim([85, 105])
plt.xlabel(r'Altitude [$' + my_diagram.axis_space.unit.print_unit() + ']$', fontsize=25)
plt.ylabel(r'H-bond ', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

## Layer

In [ ]:
plt.figure(1, figsize=(10, 7))
plt.plot(my_diagram.axis_space.value, my_diagram.mean.T[0], lw=3, label='Own')
plt.plot(my_diagram.axis_space.value, my_diagram.mean.T[1], lw=3, label='Given')
#plt.xlim([85, 105])
plt.xlabel(r'Altitude [$' + my_diagram.axis_space.unit.print_unit() + ']$', fontsize=25)
plt.ylabel(r'H-bond ', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

Here is an example to check that the layer number attribution make sens. 

The layer attribution is made initially over 6 layers (6 for the bottom interface, 1 for the bulk-like part and 6 for the upper interface). The obtained density diagram is plotted above and stored as `L_density_layer` (a list with 6+1+6 = 13 elements)

For the H-bond diagram, the layer attribution is made over 4 layers. The 4 layers defined in this diagrams should correspond to the 4 outter one of the  `L_density_layer` one (in terms of population). To check this, the code bellow reasign the `L_density_layer` using only 4 layer.

In [ ]:
# To load 
MT_name = 'Water_TIP4P2005'
name_diagram = 'hbond_layer_Water_TIP4P2005'
my_diagram = frog_data_analysis.return_diagram(GP, L_moleculetype_result, MT_name, name_diagram)

plt.figure(1, figsize=(10, 7))
plt.title('Population using 6 layers', fontsize=25)
plt.plot(L_density_layer_x, L_density_layer, lw=3)
plt.xlabel(r'Layer', fontsize=25)
plt.ylabel(r'Population', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)


plt.figure(2, figsize=(10, 7))
plt.title('Population using 4 layers', fontsize=25)
plt.plot(my_diagram.axis_space.value, my_diagram.axis_population, lw=3)
plt.xlabel(r'Layer', fontsize=25)
plt.ylabel(r'Population', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

plt.figure(3, figsize=(10, 7))
plt.title('Population using 4 or 6 layers', fontsize=25)
plt.plot(my_diagram.axis_space.value, my_diagram.axis_population, lw=3)
plt.plot(L_density_layer_x, L_density_layer, lw=3)
plt.xlabel(r'Layer', fontsize=25)
plt.ylabel(r'Population', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

In [ ]:
L_new_layer_x = [k for k in range(-4, 5, 1)] # the values goes from -4 to +4
L_new_density_layer = np.zeros(9) 

trotter = 0
for k in range(0, 4, 1): # bottom interface 
    L_new_density_layer[k] = L_density_layer[trotter] # the most outter layer of the original density correspond to the new layers 
    trotter +=1
    
for k in range(-2, 3, 1):
    L_new_density_layer[4] += L_density_layer[trotter] # the layers called -2 to 2 in the original density is know refers as 0 (bulk phase). In other words, the inner layer are now assigned to the bulk-like phase.
    trotter += 1
    
for k in range(5, 9, 1): # upper interface
    L_new_density_layer[k] = L_density_layer[trotter] 
    trotter += 1
               
plt.figure(1, figsize=(10, 7))
plt.plot(L_new_layer_x, L_new_density_layer, label = 'Reconstructed Population', lw=3)
plt.plot(my_diagram.axis_space.value, my_diagram.axis_population, label = 'Hbond Population', lw=3)
plt.xlabel(r'Layer', fontsize=25)
plt.ylabel(r'Population', fontsize=25)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.legend(fontsize=25)
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)
print('L_new_density_layer-my_diagram.axis_population:', L_new_density_layer-my_diagram.axis_population)